# Intialisation Steps
Connecting to Snowflake

## Importing required modules

In [1]:
import pandas as pd
from drg_connect import Snowflake
import qgrid 
from datetime import timedelta, datetime
import math

import warnings
warnings.filterwarnings('ignore')

## Snowflake connection parameter

In [2]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW',database='SANDBOX_ANALYTICS',schema = 'SANDBOX')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

# ICD grouper table

## Swarali notes:
- Using level_4_short_description_icd10 variable because description includes retinal vein occlusion and macular edema

In [3]:
%%read_sql

create or replace temporary table st_rvo_me_ref1 as

select
    'dx' as cat1,
    'all_me' as cat2,
    case
        when 
            lower(level_4_long_description_icd10) like '%branch%' 
            then 'brvo'
        when 
            (lower(level_4_long_description_icd10) like '%retinal%' 
            and lower(level_4_long_description_icd10) like '%vein%'
            and lower(level_4_long_description_icd10) like '%occlusion%')
            then 'crvo'
        when
            (lower(level_4_long_description_icd10) like '%venous%'
            and lower(level_4_long_description_icd10) like '%engorgement%'
            and lower(level_4_long_description_icd10) like '%eye%')
            then 've'
        else 'other_me' 
        end as cat3,
    null as cat4,
    null as cat5,
    null as cat6,
    'icd_10' as type,
    level_4 as value,
    level_4_long_description_icd10 as description,
    current_date() as active_date,
    'Nish' as active_reason,
    null as deactive_date,
    null as deactive_reason,
    'RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER' as source
from 
    RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER
where
    (lower(level_4_long_description_icd10) like '%retinal%'
    and lower(level_4_long_description_icd10) like '%vein%'
    and lower(level_4_long_description_icd10) like '%occlusion%')
        or
    (lower(level_4_long_description_icd10) like '%venous%'
    and lower(level_4_long_description_icd10) like '%engorgement%'
    and lower(level_4_long_description_icd10) like '%eye%')
        or
    (lower(level_4_long_description_icd10) like '%macular%'
    and lower(level_4_long_description_icd10) like '%edema%'
    and lower(level_4_long_description_icd10) not like '%without%')

Query started at 04:25:01 PM GMT Daylight TimeInitiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
; Query executed in 0.22 m

,status
0,Table ST_RVO_ME_REF1 successfully created.


In [4]:
%%read_sql

create or replace temporary table st_rvo_me_ref2 as

select
    'dx' as cat1,
    'all_me' as cat2,
    case
        when 
            lower(level_4_long_description_icd10) like '%branch%' 
            then 'brvo'
        when 
            (lower(level_4_long_description_icd10) like '%retinal%' 
            and lower(level_4_long_description_icd10) like '%vein%'
            and lower(level_4_long_description_icd10) like '%occlusion%')
            then 'crvo'
        when
            (lower(level_4_long_description_icd10) like '%venous%'
            and lower(level_4_long_description_icd10) like '%engorgement%'
            and lower(level_4_long_description_icd10) like '%eye%')
            then 've'
        else 'other_me' 
        end as cat3,
    null as cat4,
    null as cat5,
    null as cat6,
    'icd_9' as type,
    icd9_mapped_codes as value,
    level_4_long_description_icd10 as description,
    current_date() as active_date,
    'Nish' as active_reason,
    null as deactive_date,
    null as deactive_reason,
    'RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER' as source
from 
    RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER
where
    ((lower(level_4_long_description_icd10) like '%retinal%'
    and lower(level_4_long_description_icd10) like '%vein%'
    and lower(level_4_long_description_icd10) like '%occlusion%')
        or
    (lower(level_4_long_description_icd10) like '%venous%'
    and lower(level_4_long_description_icd10) like '%engorgement%'
    and lower(level_4_long_description_icd10) like '%eye%')
        or
    (lower(level_4_long_description_icd10) like '%macular%'
    and lower(level_4_long_description_icd10) like '%edema%'
    and lower(level_4_long_description_icd10) not like '%without%'))
    and icd9_mapped_codes != 'No map'

Query started at 04:25:20 PM GMT Daylight Time; Query executed in 0.11 m

,status
0,Table ST_RVO_ME_REF2 successfully created.


In [17]:
%%read_sql

create or replace table st_rvo_me_ref as

select * from st_rvo_me_ref1
union
select * from st_rvo_me_ref2
order by type, value, cat3

Query started at 04:35:11 PM GMT Daylight Time; Query executed in 0.11 m

,status
0,Table ST_RVO_ME_REF successfully created.


## Export ref table to inspect

In [18]:
snow.select("select * from st_rvo_me_ref")

,cat1,cat2,cat3,cat4,cat5,cat6,type,value,description,active_date,active_reason,deactive_date,deactive_reason,source
0,dx,all_me,other_me,None,None,None,icd_10,E08311,Diabetes mellitus due to underlying condition ...,2018-07-18,Nish,None,None,RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER
1,dx,all_me,other_me,None,None,None,icd_10,E08321,Diabetes mellitus due to underlying condition ...,2018-07-18,Nish,None,None,RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER
2,dx,all_me,other_me,None,None,None,icd_10,E083211,Diabetes mellitus due to underlying condition ...,2018-07-18,Nish,None,None,RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER
3,dx,all_me,other_me,None,None,None,icd_10,E083212,Diabetes mellitus due to underlying condition ...,2018-07-18,Nish,None,None,RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER
4,dx,all_me,other_me,None,None,None,icd_10,E083213,Diabetes mellitus due to underlying condition ...,2018-07-18,Nish,None,None,RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER
5,dx,all_me,other_me,None,None,None,icd_10,E083219,Diabetes mellitus due to underlying condition ...,2018-07-18,Nish,None,None,RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER
6,dx,all_me,other_me,None,None,None,icd_10,E08331,Diabetes mellitus due to underlying condition ...,2018-07-18,Nish,None,None,RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER
7,dx,all_me,other_me,None,None,None,icd_10,E083311,Diabetes mellitus due to underlying condition ...,2018-07-18,Nish,None,None,RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER
8,dx,all_me,other_me,None,None,None,icd_10,E083312,Diabetes mellitus due to underlying condition ...,2018-07-18,Nish,None,None,RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER
9,dx,all_me,other_me,None,None,None,icd_10,E083313,Diabetes mellitus due to underlying condition ...,2018-07-18,Nish,None,None,RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER


In [19]:
snow.select("select * from st_rvo_me_ref").to_excel("out/ref.xlsx", index=False)

# Analysis plan

<img src="in/cohort.png", width=700>

# Cohort in obs. window

<img src="in/cohort.png", width=700>

In [20]:
%%read_sql

create or replace table st_rvo_obs_cohort as

select
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    claim_number,
    diagnosis,
    diagnosis_sequence,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS
where
    diagnosis in (select distinct value from st_rvo_me_ref where cat2 = 'all_me' and deactive_date is null)
    and year_of_service between '2016-01-01' and '2017-12-31'

Query started at 05:12:24 PM GMT Daylight Time; Query executed in 34.63 m

,status
0,Table ST_RVO_OBS_COHORT successfully created.


In [21]:
snow.select("select count(distinct patient_id) from st_rvo_obs_cohort where left(patient_id, 5) != 'XXX -'")

,COUNT(DISTINCT PATIENT_ID)
0,670967


## Index date

In [22]:
%%read_sql

create or replace table st_rvo_index as

select
    patient_id,
    min(year_of_service) as index_date
from
    st_rvo_obs_cohort
where
    left(patient_id, 5) != 'XXX -'
group by 1

Query started at 05:47:03 PM GMT Daylight Time; Query executed in 0.08 m

,status
0,Table ST_RVO_INDEX successfully created.


In [23]:
snow.select("select * from st_rvo_index order by index_date desc limit 5")

,patient_id,index_date
0,66t6pf964yujZC20,2017-12-31
1,myuwZoVkV7eWRMVY,2017-12-31
2,CbeUja8n47xAXTuk,2017-12-31
3,kzufQjSKaQAmz5uQ,2017-12-31
4,ZJWgdKixoDAHiibb,2017-12-31


In [24]:
snow.select("select count(patient_id) from st_rvo_index where left(patient_id, 5) = 'XXX -'")

,COUNT(PATIENT_ID)
0,0


# Continuous coverage

## Cont. cov from medical claims
<img src="in/cohort.png", width=700>

### Extract submits header

In [25]:
%%read_sql

create or replace table st_rvo_cont_med as

select
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    claim_number,
    claim_type_code,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_HEADER
where
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) in (select patient_id from st_rvo_index)
    and year_of_service between '2015-07-01' and '2018-06-30'

Query started at 05:47:12 PM GMT Daylight Time; Query executed in 7.10 m

,status
0,Table ST_RVO_CONT_MED successfully created.


### Check for duplicates

In [26]:
%%read_sql

select count(*) from st_rvo_cont_med

Query started at 05:54:18 PM GMT Daylight Time; Query executed in 0.04 m

,COUNT(*)
0,43127401


In [27]:
%%read_sql

with t1 as (select distinct * from st_rvo_cont_med)
select count(*) from t1

Query started at 05:54:21 PM GMT Daylight Time; Query executed in 0.09 m

,COUNT(*)
0,42690780


In [28]:
%%read_sql

drop table if exists st_rvo_cont_med_unique;
create or replace table st_rvo_cont_med_unique as
select distinct * from st_rvo_cont_med;
drop table st_rvo_cont_med;

Query started at 05:54:26 PM GMT Daylight Time; Query executed in 0.06 mQuery started at 05:54:30 PM GMT Daylight Time; Query executed in 0.12 mQuery started at 05:54:37 PM GMT Daylight Time; Query executed in 0.08 m

,status
0,ST_RVO_CONT_MED successfully dropped.


### Summarise medical claims

In [29]:
%%read_sql

create or replace table st_rvo_cont_med_tally as

select
    a.patient_id,
    sum(case
           when datediff(d, a.index_date, b.year_of_service) between -182 and -1
           then 1
           else 0
           end) as claim_tally_pre,
    sum(case
           when datediff(d, a.index_date, b.year_of_service) between 1 and 182
           then 1
           else 0
           end) as claim_tally_post
from
    st_rvo_index a
        join st_rvo_cont_med_unique b
            on a.patient_id = b.patient_id
group by 1

Query started at 05:54:42 PM GMT Daylight Time; Query executed in 0.14 m

,status
0,Table ST_RVO_CONT_MED_TALLY successfully created.


### Final counts

In [30]:
a = snow.select("select count(distinct patient_id) from st_rvo_cont_med_tally")
b = snow.select("select count(distinct patient_id) from st_rvo_cont_med_tally where claim_tally_pre > 0 and claim_tally_post > 0")
c = snow.select("select count(distinct patient_id) from st_rvo_cont_med_tally where claim_tally_pre > 1 and claim_tally_post > 1")
d = snow.select("select count(distinct patient_id) from st_rvo_cont_med_tally where claim_tally_pre > 2 and claim_tally_post > 2")
e = snow.select("select count(distinct patient_id) from st_rvo_cont_med_tally where claim_tally_pre > 3 and claim_tally_post > 3")
f = snow.select("select count(distinct patient_id) from st_rvo_cont_med_tally where claim_tally_pre > 4 and claim_tally_post > 4")

lst = ['pts_obs_window',
       '1_claim_pre_and_post',
       '2_claims_pre_and_post',
       '3_claims_pre_and_post',
       '4_claims_pre_and_post',
       '5_claims_pre_and_post',]

pd.concat([a,b,c,d,e,f], keys=lst)

,,COUNT(DISTINCT PATIENT_ID)
pts_obs_window,0,670967
1_claim_pre_and_post,0,529987
2_claims_pre_and_post,0,456771
3_claims_pre_and_post,0,393848
4_claims_pre_and_post,0,342434
5_claims_pre_and_post,0,297935


## Final cohort

In [31]:
%%read_sql

create or replace table st_rvo_cohort as

select
    patient_id,
    index_date
from 
    st_rvo_index
where
    patient_id in (select patient_id 
                   from st_rvo_cont_med_tally 
                   where claim_tally_pre > 1
                   and claim_tally_post > 1)

Query started at 05:54:58 PM GMT Daylight Time; Query executed in 0.07 m

,status
0,Table ST_RVO_COHORT successfully created.


In [32]:
snow.select("select count(distinct patient_id) from st_rvo_cohort")

,COUNT(DISTINCT PATIENT_ID)
0,456771


# Raw data extracts

Pull all diagnoses data for the cohort of patients from July 1st 2015, to June 30th 2018
<img src="in/cohort.png", width=700>

## Claim header

In [33]:
%%read_sql

create or replace table st_rvo_header_all as 

select
     'XXXXXXXXXXXXXXXX' as patient_id,
     claim_number,
     received_date,
     claim_type_code,
     statement_from,
     statement_to,
     min_service_from,
     max_service_to,
     total_charge,
     total_allowed,
     drg_code,
     type_bill,
     admission_date,
     admit_type_code,
     admit_src_code,
     discharge_hour,
     discharge_status,
     new_medicare_source,
     year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_HEADER
where
     claim_number in (select distinct claim_number 
                      from st_rvo_cont_med_unique
                      where patient_id in (select patient_id
                                           from st_rvo_cohort))

Query started at 05:55:04 PM GMT Daylight Time; Query executed in 4.24 m

,status
0,Table ST_RVO_HEADER_ALL successfully created.


In [34]:
%%read_sql

 begin;
 update st_rvo_header_all a
    set a.patient_id = b.patient_id
   from st_rvo_cont_med_unique b
  where a.claim_number = b.claim_number;
commit;

Query started at 05:59:19 PM GMT Daylight Time; Query executed in 0.04 mQuery started at 05:59:22 PM GMT Daylight Time; Query executed in 0.41 mQuery started at 05:59:46 PM GMT Daylight Time; Query executed in 0.05 m

,status
0,Statement executed successfully.


In [35]:
%%read_sql

select
    count(patient_id),
    count(distinct patient_id),
    sum(case
           when patient_id = 'XXXXXXXXXXXXXXXX'
           then 1
           else 0
           end) as null_pt_id,
    sum(case
           when left(patient_id, 5) = 'XXX -'
           then 1
           else 0
           end) as faulty_pt_id
from st_rvo_header_all

Query started at 05:59:49 PM GMT Daylight Time; Query executed in 0.06 m

,COUNT(PATIENT_ID),COUNT(DISTINCT PATIENT_ID),null_pt_id,faulty_pt_id
0,38892014,456771,0,0


## Diagnoses

In [36]:
%%read_sql

create or replace table st_rvo_dx_all as

select
    'XXXXXXXXXXXXXXXX' as patient_id,
    claim_number,
    diagnosis,
    diagnosis_sequence,
    statement_from,
    statement_to,
    min_service_from,
    max_service_to,
    received_date,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS
where
    claim_number in (select distinct claim_number
                     from st_rvo_header_all)

Query started at 05:59:52 PM GMT Daylight Time; Query executed in 6.11 m

,status
0,Table ST_RVO_DX_ALL successfully created.


In [37]:
%%read_sql

begin;
update st_rvo_dx_all a
set a.patient_id = b.patient_id
from st_rvo_header_all b
where a.claim_number = b.claim_number

Query started at 06:05:59 PM GMT Daylight Time; Query executed in 0.06 mQuery started at 06:06:03 PM GMT Daylight Time; Query executed in 0.51 m

,number of rows updated,number of multi-joined rows updated
0,135959491,3446899


In [38]:
%%read_sql

select
    count(patient_id),
    count(distinct patient_id),
    sum(case
           when patient_id = 'XXXXXXXXXXXXXXXX'
           then 1
           else 0
           end) as null_pt_id,
    sum(case
           when left(patient_id, 5) = 'XXX -'
           then 1
           else 0
           end) as faulty_pt_id
    
from st_rvo_dx_all

Query started at 06:06:34 PM GMT Daylight Time; Query executed in 0.07 m

,COUNT(PATIENT_ID),COUNT(DISTINCT PATIENT_ID),null_pt_id,faulty_pt_id
0,135959491,456771,0,0


## Patient raw

In [39]:
%%read_sql

create or replace table st_rvo_pat_all as

select
    'XXXXXXXXXXXXXXXX' as patient_id,
    claim_number,
    patient_gender,
    patient_dob
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PATIENT
where
    claim_number in (select distinct claim_number
                     from st_rvo_header_all)

Query started at 06:06:38 PM GMT Daylight Time; Query executed in 2.63 m

,status
0,Table ST_RVO_PAT_ALL successfully created.


In [40]:
%%read_sql

 begin;
 update st_rvo_pat_all a
    set a.patient_id = b.patient_id
   from st_rvo_header_all b
  where a.claim_number = b.claim_number;
commit;

Query started at 06:09:16 PM GMT Daylight Time; Query executed in 0.06 mQuery started at 06:09:19 PM GMT Daylight Time; Query executed in 0.41 mQuery started at 06:09:44 PM GMT Daylight Time; Query executed in 0.05 m

,status
0,Statement executed successfully.


In [41]:
%%read_sql

create or replace table st_rvo_pat_all_unique as

select
    patient_id,
    min(patient_dob) as dob,
    min(patient_gender) as gender
from
    st_rvo_pat_all
group by 1

Query started at 06:09:47 PM GMT Daylight Time; Query executed in 0.16 m

,status
0,Table ST_RVO_PAT_ALL_UNIQUE successfully created.


In [42]:
%%read_sql

select
    count(patient_id),
    count(distinct patient_id),
    sum(case
           when patient_id = 'XXXXXXXXXXXXXXXX'
           then 1
           else 0
           end) as null_pt_id,
    sum(case
           when left(patient_id, 5) = 'XXX -'
           then 1
           else 0
           end) as faulty_pt_id
    
from st_rvo_pat_all_unique

Query started at 06:09:56 PM GMT Daylight Time; Query executed in 0.09 m

,COUNT(PATIENT_ID),COUNT(DISTINCT PATIENT_ID),null_pt_id,faulty_pt_id
0,456771,456771,0,0


# Master table

## Aggregate comorbidity Dx

In [43]:
snow.select("select distinct cat3 from st_rvo_me_ref")

,cat3
0,other_me
1,crvo
2,ve
3,brvo


In [54]:
%%read_sql

create or replace table st_rvo_comorbidites_agg as 

select
    a.patient_id,
    1 as all_me,
    max(case
           when b.diagnosis in (select distinct value
                                from st_rvo_me_ref
                                where (cat3 = 'crvo' 
                                or cat3 = 'brvo'
                                or cat3 = 've')
                                and deactive_date is null)
           and datediff(d, a.index_date, b.year_of_service) between -182 and 182                   
           then 1
           else 0
           end) as rvo_and_ve,
    max(case
           when b.diagnosis in (select distinct value
                                from st_rvo_me_ref
                                where cat3 = 'crvo' 
                                and deactive_date is null)
           and datediff(d, a.index_date, b.year_of_service) between -182 and 182                   
           then 1
           else 0
           end) as crvo,
    max(case
           when b.diagnosis in (select distinct value
                                from st_rvo_me_ref
                                where cat3 = 'brvo'
                                and deactive_date is null)
           and datediff(d, a.index_date, b.year_of_service) between -182 and 182                   
           then 1
           else 0
           end) as brvo,
    max(case
           when b.diagnosis in (select distinct value
                                from st_rvo_me_ref
                                where cat3 = 've'
                                and deactive_date is null)
           and datediff(d, a.index_date, b.year_of_service) between -182 and 182                   
           then 1
           else 0
           end) as ve
from
    st_rvo_cohort a
        join st_rvo_dx_all b
            on a.patient_id = b.patient_id
group by 1

Query started at 06:19:50 PM GMT Daylight Time; Query executed in 0.49 m

,status
0,Table ST_RVO_COMORBIDITES_AGG successfully cre...


In [55]:
snow.select("select * from st_rvo_comorbidites_agg limit").to_excel("out/results.xlsx", index=False)

## Patient age

In [45]:
%%read_sql

create or replace table st_rvo_patient_demographics as 

select
    a.patient_id,
    b.gender,
    datediff(yy, b.dob, a.index_date) as age,
    case
        when datediff(yy, b.dob, a.index_date) between 0 and 4 then '00-04'
        when datediff(yy, b.dob, a.index_date) between 5 and 9 then '05-09'
        when datediff(yy, b.dob, a.index_date) between 10 and 14 then '10-14'
        when datediff(yy, b.dob, a.index_date) between 15 and 19 then '15-19'
        when datediff(yy, b.dob, a.index_date) between 20 and 24 then '20-24'
        when datediff(yy, b.dob, a.index_date) between 25 and 29 then '25-29'
        when datediff(yy, b.dob, a.index_date) between 30 and 34 then '30-34'
        when datediff(yy, b.dob, a.index_date) between 35 and 39 then '35-39'
        when datediff(yy, b.dob, a.index_date) between 40 and 44 then '40-44'
        when datediff(yy, b.dob, a.index_date) between 45 and 49 then '45-49'
        when datediff(yy, b.dob, a.index_date) between 50 and 54 then '50-54'
        when datediff(yy, b.dob, a.index_date) between 55 and 59 then '55-59'
        when datediff(yy, b.dob, a.index_date) between 60 and 64 then '60-64'
        when datediff(yy, b.dob, a.index_date) between 65 and 69 then '65-69'
        when datediff(yy, b.dob, a.index_date) between 70 and 74 then '70-74'
        when datediff(yy, b.dob, a.index_date) between 75 and 79 then '75-79'
        when datediff(yy, b.dob, a.index_date) between 80 and 84 then '80-84'
        when datediff(yy, b.dob, a.index_date) between 85 and 110 then '85+'
        else null
        end as age_cat
from
    st_rvo_cohort a
        join st_rvo_pat_all_unique b
            on a.patient_id = b.patient_id

Query started at 06:10:25 PM GMT Daylight Time; Query executed in 0.12 m

,status
0,Table ST_RVO_PATIENT_DEMOGRAPHICS successfully...


## Master table

In [46]:
snow.drop_table("st_rvo_master_table")

DROP TABLE IF EXISTS sandbox_analytics.sandbox.st_rvo_master_table;
Table sandbox.st_rvo_master_table dropped!


In [56]:
%%read_sql

create or replace table st_rvo_master_table as

select
    a.patient_id,
    null as wt,
    a.gender,
    a.age,
    a.age_cat,
    b.all_me,
    b.rvo_and_ve,
    b.crvo,
    b.brvo,
    b.ve
from
    st_rvo_patient_demographics a
        join st_rvo_comorbidites_agg b
            on a.patient_id = b.patient_id

Query started at 06:23:48 PM GMT Daylight Time; Query executed in 0.07 m

,status
0,Table ST_RVO_MASTER_TABLE successfully created.


In [63]:
snow.select("select * from st_rvo_master_table limit 50")

,patient_id,wt,gender,age,age_cat,all_me,rvo_and_ve,crvo,brvo,ve
0,0/jLuyj+G851tO4O,None,F,54,50-54,1,0,0,0,0
1,ae4p3ropKTQHtjPf,None,M,41,40-44,1,0,0,0,0
2,FSyTCmM36YDsj/kA,None,M,41,40-44,1,0,0,0,0
3,gd1WtARQfAX4+GG/,None,F,35,35-39,1,0,0,0,0
4,P+mGnzzQWgdnJ2vU,None,F,82,80-84,1,0,0,0,0
5,nLvcAu0ZprHsx2Qu,None,M,66,65-69,1,0,0,0,0
6,UcngkH1Zldpya1lJ,None,F,80,80-84,1,0,0,0,0
7,J7/jmcFU3ghF6R6G,None,M,75,75-79,1,0,0,0,0
8,U9RWnE7cCwrHGF/0,None,F,74,70-74,1,0,0,0,0
9,b08wl3oHCf2ynyr0,None,M,77,75-79,1,1,0,1,0


# Summary results

In [59]:
%%read_sql

select
    --age_cat,
    sum(all_me) as me_pts,
    sum(rvo_and_ve) as rvo_and_ve_pts,
    round(sum(rvo_and_ve)/sum(all_me)*100, 2) as rvo_and_ve_in_all_me,
    sum(crvo) as crvo_pts,
    round(sum(crvo)/sum(rvo_and_ve)*100, 2) as crvo_in_rvo_and_ve,
    sum(brvo) as brvo_pts,
    round(sum(brvo)/sum(rvo_and_ve)*100, 2) as brvo_in_rvo_and_ve,
    sum(ve) as ve_pts,
    round(sum(ve)/sum(rvo_and_ve)*100, 2) as ve_in_rvo_and_ve
from
    st_rvo_master_table
--group by 1
--order by 1

Query started at 06:24:17 PM GMT Daylight Time; Query executed in 0.05 m

,me_pts,rvo_and_ve_pts,rvo_and_ve_in_all_me,crvo_pts,crvo_in_rvo_and_ve,brvo_pts,brvo_in_rvo_and_ve,ve_pts,ve_in_rvo_and_ve
0,456771,141962,31.08,62653,44.13,86484,60.92,1966,1.38


In [61]:
%%read_sql

select
    age_cat,
    sum(all_me) as me_pts,
    sum(rvo_and_ve) as rvo_and_ve_pts,
    round(sum(rvo_and_ve)/sum(all_me)*100, 2) as rvo_and_ve_in_all_me,
    sum(crvo) as crvo_pts,
    round(sum(crvo)/sum(rvo_and_ve)*100, 2) as crvo_in_rvo_and_ve,
    sum(brvo) as brvo_pts,
    round(sum(brvo)/sum(rvo_and_ve)*100, 2) as brvo_in_rvo_and_ve,
    sum(ve) as ve_pts,
    round(sum(ve)/sum(rvo_and_ve)*100, 2) as ve_in_rvo_and_ve
from
    st_rvo_master_table
group by 1
order by 1

Query started at 06:25:07 PM GMT Daylight Time; Query executed in 0.08 m

,age_cat,me_pts,rvo_and_ve_pts,rvo_and_ve_in_all_me,crvo_pts,crvo_in_rvo_and_ve,brvo_pts,brvo_in_rvo_and_ve,ve_pts,ve_in_rvo_and_ve
0,00-04,30,10,33.33,7,70.00,1,10.00,2,20.00
1,05-09,48,16,33.33,8,50.00,4,25.00,4,25.00
2,10-14,112,31,27.68,12,38.71,4,12.90,15,48.39
3,15-19,244,71,29.10,33,46.48,13,18.31,29,40.85
4,20-24,680,112,16.47,77,68.75,30,26.79,14,12.50
5,25-29,2084,216,10.36,151,69.91,60,27.78,17,7.87
6,30-34,3610,360,9.97,244,67.78,122,33.89,22,6.11
7,35-39,5351,573,10.71,335,58.46,251,43.80,31,5.41
8,40-44,8972,954,10.63,517,54.19,483,50.63,29,3.04
9,45-49,17025,1950,11.45,905,46.41,1104,56.62,59,3.03
